In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df=pd.read_csv("Documents/Materials.csv",encoding = "ISO-8859-1")
pd.set_option('display.max_colwidth', -1)
print('The shape: %d x %d' % df.shape)
df.head()

The shape: 193379 x 10


,Unnamed: 0,DEPT,STORE_CODE,PREFIX_MTL_CODE,MTL_CODE,LOCATION_CODE,MTL_DESC,MAT_DESC,UNIT,CURRENT_RATE
0,1,206,2,TR,4670263510,M/SHED R-8 KE SAMNE,CU DET,CU DET,NO,7138.60
1,2,209,3,SG,4900107255,KA-40,RELAY VAGM,RELAY VAGM,NO,5420.33
2,3,209,3,SG,4900107280,MR,RELAY VAJH-13,RELAY VAJH-13,NO,1300.93
3,4,209,3,SG,4900107298,KB-48,RELAY VAX31,RELAY VAX31,NO,4428.94
4,5,209,3,SG,4900107301,MR,RELAY DTH31,RELAY DTH31,NO,36412.03


In [3]:
df2=df[["MTL_CODE","MAT_DESC","CURRENT_RATE","UNIT"]]
df1=df[["MTL_CODE","MAT_DESC"]]

In [4]:
df1.dropna(how="any",axis=0,inplace=True)
df2.dropna(how="any",axis=0,inplace=True)
df.dropna(how="any",axis=0,inplace=True)

C:\Users\6031811test.BHELBPL\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\6031811test.BHELBPL\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
df.shape

(182683, 10)

In [6]:
names=df

In [7]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,.-/-]|\sBD|[0-9]|[""]|[(+*)]|\s|[\(\[].*?[\)\]]|(".*?\((.*?)\))|[X]| ?\([^)]+\)',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "McDonalds":')
ngrams('McDo1n""a-.l,(EE223)dX[] s')

All 3-grams in "McDonalds":


['McD', 'cDo', 'Don', 'ona', 'nal', 'alE', 'lEE', 'EEd', 'Eds']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
company_names = names['MAT_DESC'].tolist()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)

In [9]:
print(tf_idf_matrix[0])

# Check if this makes sense:

ngrams('CU-DET')

  (0, 2378)	0.59329542643255
  (0, 11686)	0.5906284261076192
  (0, 2670)	0.5469539279023922


['CUD', 'UDE', 'DET']

In [10]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [11]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(),200,0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 54.635146141052246


In [12]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,'right_side': right_side,'similairity': similairity,'Material Codes':df["MTL_CODE"],'CURRENT_RATE':df["CURRENT_RATE"],'UNIT':df["UNIT"]})

In [13]:
matches_df = get_matches_df(matches,company_names,top=182683)
#matches_df = matches_df[matches_df['similairity'] < 0.99999]
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
matches_df

,left_side,right_side,similairity,Material Codes,CURRENT_RATE,UNIT
0,CU DET,CU DET,1.0,4670263510,7138.60,NO
1,CU DET,CU DET,1.0,4900107255,5420.33,NO
2,CU DET,CU DET,1.0,4900107280,1300.93,NO
3,CU DET,CU DET,1.0,4900107298,4428.94,NO
4,CU DET,CU DET,1.0,4900107301,36412.03,NO
5,CU DET,CU DET,1.0,4900107310,3923.89,NO
6,CU DET,CU DET,1.0,4900107328,3685.39,NO
7,CU DET,CU DET,1.0,4900107344,263.41,NO
8,CU DET,CU DET,1.0,4900067240,4858.09,NO
9,CU DET,CU DET,1.0,4900067250,4944.46,NO


In [14]:
matches_df.shape

(182683, 6)

In [15]:
matches_df.to_excel (r'D:\FINAL_PROJECTkai.xlsx')